In [ ]:
import os
from groundedvision.config import RAW_DATA_DIR, PROCESSED_DATA_DIR
from groundedvision.equirect_to_cubemap import *
from tqdm import tqdm

from loguru import logger

In [ ]:
path = f'{PROCESSED_DATA_DIR}/matched_pairs_json'
list_of_aligned_directories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

In [ ]:
from pathlib import Path
import numpy as np

for folder_path in list_of_aligned_directories:
    input_folder_path = f"{PROCESSED_DATA_DIR}/matched_pairs_json/{folder_path}"
    for image_path in os.listdir(input_folder_path):
        output_path = f"{PROCESSED_DATA_DIR}/matched_pairs_json_fov_120/{folder_path}"
        full_path = os.path.join(input_folder_path, image_path)
        path_full_path = Path(full_path)
        # check if the image patch contains
        if "_aligned.JPG" in full_path:
            logger.info(f"Processing {full_path}")
            img_in = np.array(Image.open(full_path).convert('RGB'))
            faces = convert_back(img_in, 512)
            face_names = ['back', 'left', 'front', 'right', 'top', 'bottom']
            logger.info("  Saving output faces...")
            for i, face in enumerate(faces):
                file_face_output_dir = f"{output_path}/cubemap"
                Path(file_face_output_dir).mkdir(parents=True, exist_ok=True)
                file_face_path = f"{file_face_output_dir}/{path_full_path.stem}_{i}.jpg"

                face.convert('RGB').save(file_face_path, 'JPEG', quality=100)
                logger.info(f"    Saved: {file_face_path} ({face_names[i]})")

            collage = create_collage(faces, 512)
            collage_filename = f"{file_face_output_dir}/{path_full_path.stem}_collage.jpg"
            collage.save(collage_filename, 'JPEG', quality=100)